The ESC403 cluster comes with [Catherine Devlin's `%sql`-magic for IPython][1]; this allows you to run SQL queries from the IPython notebook, and intermix them with Python code.

[1]: https://github.com/catherinedevlin/ipython-sql

Before we can use the `%sql` syntax, two steps must be taken:

* Load the IPython-SQL bridge code

In [2]:
%load_ext sql


* Connect to an actual database; this must be the first `%sql` statement (the funny `rmurri@/lustre` syntax is correct and means "connect to the PostGreSQL DB named `lustre` running on *this* host as user `rmurri`"  -- please replace `rmurri` with your local user name):

In [3]:
%sql postgresql://muesim@/lustre


u'Connected: muesim@lustre'

Now you can run 1-line SQL queries by prefixing them with `%sql`:

In [4]:
%sql select * from lustre limit 5;

5 rows affected.


usr,grp,atime,mtime,blksize,size,path
usr356,i5105,1360399005,1368700859,4,2248,/scratch/bioc/usr356/param/ZINC04992888.rtf
usr356,i5105,1360387170,1368792474,8,6640,/scratch/bioc/usr356/param/ZINC71832434.prm
usr356,i5105,1360416215,1368785748,8,4480,/scratch/bioc/usr356/param/ZINC71850771.prm
usr356,i5105,1360404430,1368792486,4,2556,/scratch/bioc/usr356/param/ZINC34729086.rtf
usr356,i5105,1360420861,1368774402,4,2065,/scratch/bioc/usr356/param/ZINC45891658.rtf


It is also possible to run multi-line (or multiple) SQL queries by using the `%%sql` syntax instead.  Note that in this case the SQL instructions *must not* be on the same line as the `%%sql` magic marker:

In [5]:
%%sql
select count(*) from lustre;
select distinct count(usr) from lustre;
select distinct count(grp) from lustre;

1 rows affected.
1 rows affected.
1 rows affected.


count
30356776


The `%%time` magic prints the time taken to evaluate a cell (which comes handy when doing performance comparisons):

In [6]:
%%time

import time
time.sleep(5)

CPU times: user 0 ns, sys: 4.02 ms, total: 4.02 ms
Wall time: 5.01 s


----

**Note:** to keep running times low, we will be using table `lustre_sample` throughout, which contains a sample of 5% the rows of the original `lustre` table.

## 1. How many files are there in the `lustre` dataset

In [7]:
%time %sql select count(*) from lustre_sample;

1 rows affected.
CPU times: user 3.78 ms, sys: 0 ns, total: 3.78 ms
Wall time: 181 ms


count
1519053


Since we know that all values in the `path` column are distinct, we can omit the `DISTINCT` keyword. Does it gain us some performance?

In [8]:
%time %sql select count(distinct path) from lustre_sample;

1 rows affected.
CPU times: user 5.99 ms, sys: 0 ns, total: 5.99 ms
Wall time: 1min 7s


count
1519053


## 2. How many distinct users own files in the dataset?

In [9]:
%sql select count(distinct usr) from lustre_sample;

1 rows affected.


count
40


## 3. What user has the largest number of files?

In [10]:
%sql select distinct usr, count(path) as cnt from lustre_sample group by usr order by cnt desc limit 3;

3 rows affected.


usr,cnt
usr356,602120
usr388,485650
usr264,189651


## 4. Which is/are the most-recently accessed file(s)?

The `atime` field stores the last-access time in UNIX "epoch" format.
The *maximum* `atime` value is the most recent one::

In [11]:
%sql select max(atime) from lustre_sample;

1 rows affected.


max
1393664299


Then you should of course select the path(s) with that access time::

In [12]:
%sql select path from lustre_sample where atime=(select max(atime) from lustre_sample);

1 rows affected.


path
/scratch/bioc/usr384/TD-RS/run08/equi/output/restart.xsc


If we already know the *number* of files with maximum `atime`, we can
use a single query to get their names::

In [13]:
%sql select path from lustre_sample order by atime desc limit 1;

1 rows affected.


path
/scratch/bioc/usr384/TD-RS/run08/equi/output/restart.xsc


Just in case you wonder: `atime=max(atime)` is not a valid `WHERE` clause::

In [14]:
%sql select path from lustre_sample where atime=max(atime);

(psycopg2.ProgrammingError) aggregate functions are not allowed in WHERE
LINE 1: select path from lustre_sample where atime=max(atime);
                                                   ^
 [SQL: 'select path from lustre_sample where atime=max(atime);']


## 5. Let's say that $T_0$ is the time the most-recently accessed file was last read. How many files were not read in the month preceding $T_0$?

We know that T0=1393671122. Now it's easy to select paths accessed in
the given range::

In [15]:
%sql select count(path) from lustre_sample where atime > (1393664299 - 30*24*60*60)

1 rows affected.


count
26128


## 6. Is it possible to convert fields atime and mtime to PostgreSQL's TIMESTAMP type?

Yes, it *is* possible to alter a SQL table definition after the table has been created.

We shall now show how to (1) create a new table, (2) populate it, then (3) alter the definition and (4) fill the new column with values.

In [16]:
%sql create table times as (select * from lustre_sample);

(psycopg2.ProgrammingError) relation "times" already exists
 [SQL: 'create table times as (select * from lustre_sample);']


In [17]:
%sql alter table times add column atime_timestamp timestamp;

(psycopg2.ProgrammingError) column "atime_timestamp" of relation "times" already exists
 [SQL: 'alter table times add column atime_timestamp timestamp;']


In [18]:
%sql update times set atime_timestamp=to_timestamp(atime);

1519053 rows affected.


[]

Show some data from the table we created::

In [19]:
%sql select atime, atime_timestamp, path from times limit 3;

3 rows affected.


atime,atime_timestamp,path
1390288156,2014-01-21 07:09:16,/scratch/bioc/usr296/actin/protein_reid-unbound_5/distances/wordom_dist_reidi_res_10.xvg
1390290448,2014-01-21 07:47:28,/scratch/bioc/usr296/actin/protein_reid-unbound_5/distances/wordom_dist_reidi_ring_res_121.xvg
1390292628,2014-01-21 08:23:48,/scratch/bioc/usr296/actin/protein_reid-unbound_5/distances/wordom_dist_reidi_ring_res_233.xvg


## 7. Can you count the number of files in a given directory?

Yes, using SQL's `like` string matching operator, which allows any
part of a string to be matched by the `%` character (i.e., just like
`*` for file names)::

In [4]:
%sql select count(path) from lustre_sample where path like '/scratch/bioc/usr384/%'

1 rows affected.


count
129


## 8. Can you find the directory that holds the largest number of files? 

(psycopg2.ProgrammingError) syntax error at or near "split_part"
LINE 1: split_part('Hello.Pendejo', '.',2)
        ^
 [SQL: "split_part('Hello.Pendejo', '.',2)"]


## 9. Can you find the directory tree that holds the largest number of files?

Yes or no? *(and why?)*